# Agent

In [ ]:
import wikipedia
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents import tool
from langchain_core.pydantic_v1 import BaseModel, Field

from dotenv import load_dotenv

load_dotenv()
class WikipediaQuery(BaseModel):
    query: str = Field(description="The query to search on wikipedia.")
    
class RestaurantSearchQuery(BaseModel):
    cuisine: str = Field(description="The cuisine to search for.")
    location: str = Field(description="The location to search for restaurants.",default="Berlin")
    
@tool(args_schema=WikipediaQuery)
def wiki_search(query: str) -> str:
    """Search Wikipedia for a query."""
    try:
        page_titles = wikipedia.search(query)
    except:
        return "It is currently busy."
    
    if len(page_titles) == 0:
        return "No Wikipedia search results found."

    summaries = []
    for page_title in page_titles[:2]:
        try:
            wiki_page = wikipedia.page(page_title, auto_suggest=False)
            summaries.append(f"Title: {page_title}\nSummary: {wiki_page.summary}")

        except:
            pass
    if not summaries:
        return "No Wikipedia search results found."
    return """\n\n-------------------------------PAGE-------------------------------\n\n""".join(summaries)

@tool(args_schema=RestaurantSearchQuery)
def restaurant_search(cuisine: str,location: str="Berlin") -> str:
    """Search for restaurants in a location and cuisine."""
    return f"Searching for {cuisine} restaurants in {location}. You can find the best {cuisine} restaurants in {location} by going in 500m to the left and then 200m to the right. The name of the restaurant is 'The Best {cuisine} Restaurant'."

### Function Output

In [ ]:
print(wiki_search("Berlin"))

In [ ]:
print(restaurant_search("vietnamese"))

### Formating function for Openai

the format works also on other models like Mixtral

In [ ]:
format_tool_to_openai_function(wiki_search)

# Testing model with function

In [ ]:
from langchain.chat_models import ChatOpenAI
import os
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_functions
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0,api_key=os.getenv("OPENAI_KEY"))

In [ ]:
llm_with_function = llm.bind(functions=[format_tool_to_openai_function(func) for func in [wiki_search,restaurant_search]])

## Chattemplate
Please look to the link below to understand, how I build `ChatPromptTemplate`
https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{question}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
chain = prompt | llm_with_function | OpenAIFunctionsAgentOutputParser()

### first Example

In [ ]:
output = chain.invoke({"question": "Where is Berlin?","agent_scratchpad":[]})

In [ ]:
output

### the output here ie. tool and tool response are `intermediate_steps`

In [ ]:
output.tool

In [ ]:
tool_response = wiki_search(output.tool_input)

In [ ]:
print(tool_response)

### Second Example

In [ ]:
output = chain.invoke({"question": "Where can I find a good vietnamese reustaurant in Berlin","agent_scratchpad":[]})

In [ ]:
print(restaurant_search(output.tool_input))

### `function_messages` is `intermediate_steps` response

In [ ]:
function_messages=format_to_openai_functions([(output, restaurant_search(output.tool_input))])

In [ ]:
function_messages

### Add the ``AIMessage`` and ``FunctionMessage`` back to `agent_scratchpad`, they are the response from intermediate  step

In [ ]:
agent_finish = chain.invoke({
    "question": "Where can I find a good vietnamese reustaurant in Berlin",
    "agent_scratchpad": function_messages
})

In [ ]:
agent_finish.return_values["output"]

# Create an Agent

In [ ]:
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda
from langchain.agents import AgentExecutor

from operator import itemgetter

In [ ]:
agent_chain = (RunnableParallel({"question": itemgetter("question"),
                          "agent_scratchpad":itemgetter("intermediate_steps") | RunnableLambda(format_to_openai_functions)}) | 
                          prompt | 
                          llm_with_function | 
                          OpenAIFunctionsAgentOutputParser())

In [ ]:
agent_executor = AgentExecutor(agent=agent_chain, tools=[wiki_search,restaurant_search], verbose=False)

In [ ]:
agent_executor.invoke({"question": "Where can I find a good vietnamese reustaurant in Berlin"})

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,
                                  memory_key="history",
                                  input_key="question", # key name from input to add to memory
                                  output_key="output")# key name from output to add to memory

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent_chain_with_memory = (RunnableParallel({"question":itemgetter("question"),
                            "agent_scratchpad":itemgetter("intermediate_steps") | RunnableLambda(format_to_openai_functions),
                            "history":itemgetter("history")}) |
                prompt | 
                llm_with_function | 
                OpenAIFunctionsAgentOutputParser())

In [ ]:
agent_with_memory = AgentExecutor(agent=agent_chain_with_memory, tools=[wiki_search,restaurant_search], verbose=False,memory=memory)

In [ ]:
x=agent_with_memory.invoke({"question": "Hello, my name is Long."})

In [ ]:
print(x["output"])

In [ ]:
x=agent_with_memory.invoke({"question": "What is my name?"})
print(x["output"])

In [ ]:
print(agent_with_memory.invoke({"question": "What are the things that Berlin is famous for?"})["output"])


# Tavily

for using tavily you need to run:

`pip install tavily-python`

You can create an free account.

forcing function call with function_call={"name":"tavily_search"}

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
tav = TavilySearchResults(max_results=1)

In [ ]:
tav("What are the current news from nvidia?")

In [ ]:
class TavilyQuery(BaseModel):
    query: str = Field(description="The query to search for news.")
    
@tool(args_schema=TavilyQuery)
def tavily_search(query: str) -> str:
    """Search news on tavily."""
    tavily = TavilySearchResults(max_results=3)
    docs = tavily(query)
    if not docs:
        return "No news found."
    return """\n\n-------------------------------PAGE-------------------------------\n\n""".join([doc["content"] for doc in docs])

In [ ]:
memory = ConversationBufferMemory(return_messages=True,
                                  memory_key="history",
                                  input_key="question", # key name from input to add to memory
                                  output_key="output")# key name from output to add to memory
agent_chain_with_memory = (RunnableParallel({"question":itemgetter("question"),
                            "agent_scratchpad":itemgetter("intermediate_steps") | RunnableLambda(format_to_openai_functions),
                            "history":itemgetter("history")}) |
                prompt | 
                llm.bind(functions=[format_tool_to_openai_function(func) for func in [tavily_search, wiki_search]]) | 
                OpenAIFunctionsAgentOutputParser())
agent_with_memory = AgentExecutor(agent=agent_chain_with_memory, tools=[tavily_search,wiki_search], verbose=True,memory=memory)

In [ ]:
output = agent_with_memory.invoke({"question": "What are the current news from nvidia about their share?"})

In [ ]:
output = agent_with_memory.invoke({"question": "What are the thing that Berlin are famous for?"})

In [ ]:
print(output["output"])